17. １列目の文字列の異なり
1列目の文字列の種類（異なる文字列の集合）を求めよ．確認にはsort, uniqコマンドを用いよ．

In [136]:
with open('hightemp.txt', "r") as f:
    data = f.readlines() #データを読み込んでリストにする

In [140]:
col1 = set() #1列目の集合を作成
for x in data:
    x1 = x.split('\t')
    col1.add(x1[0])

print(col1)

{'愛媛県', '群馬県', '埼玉県', '山形県', '大阪府', '千葉県', '高知県', '山梨県', '岐阜県', '和歌山県', '愛知県', '静岡県'}


In [63]:
with open('col1.txt', "r") as f1:
    data1 = f1.read()
data1 = data1.split('\n')

In [64]:
data1

['高知県',
 '埼玉県',
 '岐阜県',
 '山形県',
 '山梨県',
 '和歌山県',
 '静岡県',
 '山梨県',
 '埼玉県',
 '群馬県',
 '群馬県',
 '愛知県',
 '千葉県',
 '静岡県',
 '愛媛県',
 '山形県',
 '岐阜県',
 '群馬県',
 '千葉県',
 '埼玉県',
 '大阪府',
 '山梨県',
 '山形県',
 '愛知県',
 '']

In [ ]:
with open('col1.txt', "r") as f:
    data = f.read() #データを読み込む
data = data.replace("\n", "") #データを置換

with open('col1.txt', "w") as f: #新しいファイルに置換したデータを書き込む
    f.write(data)
        

In [69]:
with open('col1.txt', "r") as f1:
    data1 = f1.read()
data1 = data1.split('\n')

with open('col2.txt', "r") as f2:
    data2 = f2.read()
data2 = data2.split('\n')
    
with open('col.txt', "w") as f:
    for (a, b) in zip(data1, data2):
        f.write(a + '\t' + b + '\n')
                

   


In [34]:
with open('col1.txt', "w") as f1: #col1に1列目を保存
    for l in data:
        l = l.split('\t')
        f1.write(l[0] + '\n')

In [35]:
with open('col2.txt', "w") as f2: #col2に2列目を保存
    for l in data:
        l = l.split('\t')
        f2.write(l[1] + '\n')

In [23]:
data

['高知県\t江川崎\t41\t2013-08-12\n',
 '埼玉県\t熊谷\t40.9\t2007-08-16\n',
 '岐阜県\t多治見\t40.9\t2007-08-16\n',
 '山形県\t山形\t40.8\t1933-07-25\n',
 '山梨県\t甲府\t40.7\t2013-08-10\n',
 '和歌山県\tかつらぎ\t40.6\t1994-08-08\n',
 '静岡県\t天竜\t40.6\t1994-08-04\n',
 '山梨県\t勝沼\t40.5\t2013-08-10\n',
 '埼玉県\t越谷\t40.4\t2007-08-16\n',
 '群馬県\t館林\t40.3\t2007-08-16\n',
 '群馬県\t上里見\t40.3\t1998-07-04\n',
 '愛知県\t愛西\t40.3\t1994-08-05\n',
 '千葉県\t牛久\t40.2\t2004-07-20\n',
 '静岡県\t佐久間\t40.2\t2001-07-24\n',
 '愛媛県\t宇和島\t40.2\t1927-07-22\n',
 '山形県\t酒田\t40.1\t1978-08-03\n',
 '岐阜県\t美濃\t40\t2007-08-16\n',
 '群馬県\t前橋\t40\t2001-07-24\n',
 '千葉県\t茂原\t39.9\t2013-08-11\n',
 '埼玉県\t鳩山\t39.9\t1997-07-05\n',
 '大阪府\t豊中\t39.9\t1994-08-08\n',
 '山梨県\t大月\t39.9\t1990-07-19\n',
 '山形県\t鶴岡\t39.9\t1978-08-03\n',
 '愛知県\t名古屋\t39.9\t1942-08-02\n']

In [133]:
n = input("ファイル数を指定：")
n = int(n)

ファイル数を指定：5


In [134]:
with open('hightemp.txt', "r") as f:
    data = f.readlines() #データを読み込んでリストにする

l = -(-len(data) // n) #行数（全行数 /  分割ファイル数、あまりは切り上げ）
print("ファイル全体の行数:" + str(len(data)))
print("1ファイルに書き込む行数:" + str(l))



ファイル全体の行数:24
1ファイルに書き込む行数:5


In [135]:
for i in range(n): #iはファイル数、jは行数
    with open('file{}.txt'.format(i+1), "w") as f:
        for j in range(i*l, min(len(data), (i+1)*l)): #(0行からl行目の1行前まで)（l行目から2*l行の1行前まで）…
            f.write(data[j])

In [ ]:
#18. 各行を3コラム目の数値の降順にソート
#各行を3コラム目の数値の逆順で整列せよ（注意: 各行の内容は変更せずに並び替えよ）．確認にはsortコマンドを用いよ（この問題はコマンドで実行した時の結果と合わなくて
もよい）．

In [141]:
with open('hightemp.txt', "r") as f:
    data = f.readlines() #データを読み込んでリストにする

In [144]:
data

['高知県\t江川崎\t41\t2013-08-12\n',
 '埼玉県\t熊谷\t40.9\t2007-08-16\n',
 '岐阜県\t多治見\t40.9\t2007-08-16\n',
 '山形県\t山形\t40.8\t1933-07-25\n',
 '山梨県\t甲府\t40.7\t2013-08-10\n',
 '和歌山県\tかつらぎ\t40.6\t1994-08-08\n',
 '静岡県\t天竜\t40.6\t1994-08-04\n',
 '山梨県\t勝沼\t40.5\t2013-08-10\n',
 '埼玉県\t越谷\t40.4\t2007-08-16\n',
 '群馬県\t館林\t40.3\t2007-08-16\n',
 '群馬県\t上里見\t40.3\t1998-07-04\n',
 '愛知県\t愛西\t40.3\t1994-08-05\n',
 '千葉県\t牛久\t40.2\t2004-07-20\n',
 '静岡県\t佐久間\t40.2\t2001-07-24\n',
 '愛媛県\t宇和島\t40.2\t1927-07-22\n',
 '山形県\t酒田\t40.1\t1978-08-03\n',
 '岐阜県\t美濃\t40\t2007-08-16\n',
 '群馬県\t前橋\t40\t2001-07-24\n',
 '千葉県\t茂原\t39.9\t2013-08-11\n',
 '埼玉県\t鳩山\t39.9\t1997-07-05\n',
 '大阪府\t豊中\t39.9\t1994-08-08\n',
 '山梨県\t大月\t39.9\t1990-07-19\n',
 '山形県\t鶴岡\t39.9\t1978-08-03\n',
 '愛知県\t名古屋\t39.9\t1942-08-02\n']

In [156]:
sorted(data, key=lambda data: data.split('\t')[2], reverse=True)

['高知県\t江川崎\t41\t2013-08-12\n',
 '埼玉県\t熊谷\t40.9\t2007-08-16\n',
 '岐阜県\t多治見\t40.9\t2007-08-16\n',
 '山形県\t山形\t40.8\t1933-07-25\n',
 '山梨県\t甲府\t40.7\t2013-08-10\n',
 '和歌山県\tかつらぎ\t40.6\t1994-08-08\n',
 '静岡県\t天竜\t40.6\t1994-08-04\n',
 '山梨県\t勝沼\t40.5\t2013-08-10\n',
 '埼玉県\t越谷\t40.4\t2007-08-16\n',
 '群馬県\t館林\t40.3\t2007-08-16\n',
 '群馬県\t上里見\t40.3\t1998-07-04\n',
 '愛知県\t愛西\t40.3\t1994-08-05\n',
 '千葉県\t牛久\t40.2\t2004-07-20\n',
 '静岡県\t佐久間\t40.2\t2001-07-24\n',
 '愛媛県\t宇和島\t40.2\t1927-07-22\n',
 '山形県\t酒田\t40.1\t1978-08-03\n',
 '岐阜県\t美濃\t40\t2007-08-16\n',
 '群馬県\t前橋\t40\t2001-07-24\n',
 '千葉県\t茂原\t39.9\t2013-08-11\n',
 '埼玉県\t鳩山\t39.9\t1997-07-05\n',
 '大阪府\t豊中\t39.9\t1994-08-08\n',
 '山梨県\t大月\t39.9\t1990-07-19\n',
 '山形県\t鶴岡\t39.9\t1978-08-03\n',
 '愛知県\t名古屋\t39.9\t1942-08-02\n']